In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
import copy
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
from sklearn.metrics import confusion_matrix
import sys
import torchvision
import matplotlib
import pandas as pd

/home/ec2-user/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
data = pd.read_csv("new_dataset_participants/participants_dataset.csv")

factorization = pd.factorize(data['label'])
data['label'] = factorization[0]

train_idxs = np.random.choice(np.arange(8216), 6162, replace=False)
val_idxs = np.delete(np.arange(8216), train_idxs)

all_data = data.values[:8216]
train_data = data.values[train_idxs]
val_data = data.values[val_idxs]
test_data = data[8216:].values

In [4]:
# the mean was already calculated using the formulas above
mean = np.array([116.03621132941863, 119.99914624598972, 104.72793001034701]) / 255.
std = np.array([50.91575939112449, 45.65966540949138, 54.019100665502926]) / 255.

In [5]:
class PlantDataset(torch.utils.data.Dataset):
    
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, ID):
        image = Image.open("new_dataset_participants/" + self.data[ID, 1])
        label = self.data[ID, 2]
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [6]:
confusion = False

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False, confusion=confusion):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        conf_y = np.zeros(shape=(0))
        conf_val = np.zeros(shape=(0))
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                                
                # save for confusion matrix
                if phase == 'val' and confusion == True:
                    conf_y = np.hstack((conf_y, labels.cpu().numpy()))
                    conf_val = np.hstack((conf_val, preds.cpu().numpy()))

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            if phase == 'val' and confusion == True:
                print(confusion_matrix(conf_y, conf_val))
                
            if phase == 'val':
                scheduler.step(loss)
                
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, val_acc_history

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 256

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet34(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 256

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 256

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 256

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 256

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet161(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 256

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [7]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 15

# Batch size for training (change depending on how much memory you have)
batch_size = 32

val_test_batch_size = 180

# Number of epochs to train for
num_epochs = 30

# Flag for feature extracting. When False, we finetune the whole model,
# when True we only update the reshaped layer params
feature_extract = False

use_pretrained = False

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained)

# Send the model to GPU
model_ft = model_ft.to(device)

print("Initializing Datasets and Dataloaders...")

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode='max', patience=10, factor=0.7)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Transforms
train_transforms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(degrees=(0, 30), translate=(0.1, 0.25)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
        #transforms.RandomErasing(),
    ])
    
val_transforms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    
test_transforms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

# torch Dataset
train_dataset = PlantDataset(train_data, transform=train_transforms)
val_dataset = PlantDataset(val_data, transform=val_transforms)

# torch Dataloaders
train_load = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_load = torch.utils.data.DataLoader(val_dataset, batch_size = val_test_batch_size, shuffle = True)
dataloaders = {'train': train_load, 'val':val_load}

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))


/home/ec2-user/.local/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/home/ec2-user/.local/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Initializing Datasets and Dataloaders...
Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer1.2.conv1.weight
	 layer1.2.bn1.weight
	 layer1.2.bn1.bias
	 layer1.2.conv2.weight
	 layer1.2.bn2.weight
	 layer1.2.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer2.2.conv1.weight
	 layer2.2.bn1.weight
	 layer2.2.bn1.bias
	 layer2.2.conv2.weight
	 layer2.2.bn2.weight
	 layer2.2.bn2.bias
	 layer2.3.conv1.wei

In [11]:
import copy

In [16]:
model_ft2, hist = train_model(model_ft2, dataloaders, criterion, optimizer_ft, num_epochs=2, is_inception=(model_name=="inception"))


Epoch 0/1
----------
train Loss: 0.2957 Acc: 0.9034
val Loss: 0.1612 Acc: 0.9513
Epoch 1/1
----------
train Loss: 0.3054 Acc: 0.8973
val Loss: 0.1561 Acc: 0.9508
Training complete in 2m 1s
Best val Acc: 0.951315


In [15]:
all_dataset = PlantDataset(all_data, transform=train_transforms)
all_dataloader = torch.utils.data.DataLoader(all_dataset, batch_size=batch_size, shuffle=True)
all_dataloader = {'train': all_dataloader, 'val':val_load}

optimizer_retrain = optim.SGD(params_to_update, lr=0.00014, momentum=0.9)
num_epochs_retrain = 4
model_ft2, hist = train_model(model_ft2, all_dataloaders, criterion, optimizer_retrain, num_epochs=num_epochs_retrain, is_inception=(model_name=="inception"))

ValueError: optimizer got an empty parameter list

In [17]:
# %% predict for test

# torch test Dataset and Dataloader
test_dataset = PlantDataset(test_data, transform=val_transforms)
test_load = torch.utils.data.DataLoader(test_dataset, batch_size=val_test_batch_size, shuffle = False)

# Predict
pred = []
with torch.no_grad():
    model_ft2.eval()
    for inputs in test_load:
        inputs = inputs[0].to(device)
        outputs = model_ft2(inputs)
        _, preds = torch.max(outputs, 1)
        pred.append(preds.cpu().numpy())

# Save the predictions
preds = np.concatenate(pred)

In [18]:
# %% save predictions
#pred = pd.DataFrame(np.vstack([train_data[:,0],train_data[:,2]])).T
pred2 = pd.DataFrame(np.vstack([test_data[:,0], factorization[1][preds]])).T
#pred3 = pd.DataFrame(np.vstack([pred, pred2]))
pred2.to_csv('pred_resnet_34.csv', index=False, header=['ID', 'label'])